# Prova Data Science N1

## <font color='red'>Importante: a prova é individual.</font>

**Instruções:**

1. Faça download deste notebook e carregue o mesmo na sua conta do Google Colab.
2. Responda às questões propostas.
3. Crie um repositório para o notebook, "commite" o notebook neste repositório.
4. Na tarefa de avaliação do Teams, envie o link do repositório. Arquivos enviados diretamente no Teams não serão considerados.
5. Commits no repositório após a data de entrega (combinada em sala de aula) serão desconsiderados.

# Objeto da avaliação

Você irá fazer o download do arquivo de dados matches.csv do [repositório do Github](https://raw.githubusercontent.com/mdietterle/repositorio_dados/main/matches.csv)

Todas as questões/atividades serão realizadas com base neste arquivo. É importante que você responda aos questionamentos propostos o mais completo possível, com a maior quantidade de detalhes que você conseguir incluir.

Não se contente com o básico, seja criativo nas respostas, indo além do tradicional.

## **Importante!!**

Antes de começar a responder, abra o arquivo de dados e entenda o dataset. Não será fornecido um dicionário de dados, é sua tarefa analisar o dataset e "inferir" o que são as colunas e a sua organização.

## Instalação de bibliotecas necessárias

Instale neste local todas as bibliotecas necessárias para resolução da avaliação.

In [37]:
# Instalação de bibliotecas básicas
!pip install pandas plotly

## Estatísticas básicas do dataset


Carregue o arquivo de dados no notebook e exiba as características básicas do arquivo (shape, tipo de dados, dados ausentes, estatísticas básicas dos dados, etc)

In [38]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from bs4 import BeautifulSoup

file_path = '/content/matches.csv'
data = pd.read_csv(file_path)

data_cleaned = data.dropna(subset=['gf', 'ga', 'sh', 'sot'])

## Dia da semana com placares mais altos

Crie um gráfico que responda em qual dia da semana os placares foram mais altos. O conceito de "placar mais alto" é a soma dos gols feitos no jogo.

Como a quantidade de jogos não é a mesma em todos os dias da semana (finais de semana tendem a ter mais jogos que em dias da semana - o que resultaria em placares mais altos nestes dias), você deverá encontrar uma forma de equalizar esta diferença na quantidade de jogos.

A resposta deverá ser um gráfico que demonstre claramente o comparativo dos dias.

In [48]:
data_cleaned['total_goals'] = data_cleaned['gf'] + data_cleaned['ga']

games_per_day = data_cleaned['day'].value_counts()
total_goals_per_day = data_cleaned.groupby('day')['total_goals'].sum()
average_goals_per_game_day = total_goals_per_day / games_per_day

ordered_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
average_goals_per_game_day = average_goals_per_game_day.reindex(ordered_days)

fig_day_goals_line = px.line(average_goals_per_game_day.sort_index(),
                             x=average_goals_per_game_day.index,
                             y=average_goals_per_game_day.values,
                             title='Média de Gols por Jogo em Cada Dia da Semana (Equalizado por Jogos)',
                             labels={'x':'Dia da Semana', 'y':'Média de Gols por Jogo'},
                             markers=True)

fig_day_goals_line.update_yaxes(range=[0, 6])

global_average_goals = average_goals_per_game_day.mean()
fig_day_goals_line.add_hline(y=global_average_goals, line_dash="dot", line_color="green",
                             annotation_text=f'Média Global: {global_average_goals:.2f} gols',
                             annotation_position="bottom right")

max_day = average_goals_per_game_day.idxmax()
min_day = average_goals_per_game_day.idxmin()
fig_day_goals_line.add_annotation(x=max_day, y=average_goals_per_game_day[max_day],
                                  text=f'Máxima: {average_goals_per_game_day[max_day]:.2f} gols',
                                  showarrow=True, arrowhead=2)
fig_day_goals_line.add_annotation(x=min_day, y=average_goals_per_game_day[min_day],
                                  text=f'Mínima: {average_goals_per_game_day[min_day]:.2f} gols',
                                  showarrow=True, arrowhead=2)

fig_day_goals_line.update_traces(marker=dict(size=10, color='rgb(26, 118, 255)', symbol='circle'))

fig_day_goals_line.show()


## Capitães

Quantos capitães diferentes cada equipe teve durante a temporada?

Responda em forma de um gráfico comparativo, apresentando os dados o mais claramente possível.

In [51]:
captains_per_team = data_cleaned.groupby('team')['captain'].nunique()
captains_per_team_sorted = captains_per_team.sort_values(ascending=False)

fig_captains = px.bar(captains_per_team_sorted,
                      x=captains_per_team_sorted.index,
                      y=captains_per_team_sorted.values,
                      title='Número de Capitães Diferentes por Equipe na Temporada',
                      labels={'x':'Equipe', 'y':'Número de Capitães Diferentes'},
                      text=captains_per_team_sorted.values,
                      color=captains_per_team_sorted.values,
                      color_continuous_scale='Viridis')

fig_captains.update_layout(
    xaxis_title='Equipe',
    yaxis_title='Número de Capitães Diferentes',
    xaxis_tickangle=-45,
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    coloraxis_colorbar=dict(
        title='Número de Capitães',
        tickvals=[0, max(captains_per_team_sorted)],
        ticktext=['Menos', 'Mais']
    ),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

fig_captains.update_traces(texttemplate='%{text}', textposition='outside')

fig_captains.show()


## Formação Tática

Qual a formação tática mais comum para cada equipe de acordo com o resultado obtido (Ex: para vitórias, a formação mais comum é X, para empates Y e para Z em derrotas).

**Importante** responda de forma que seja possível fazer comparativos entre equipes.

Responda com um gráfico.

In [55]:
formation_counts = data_cleaned.groupby(['team', 'result', 'formation']).size().reset_index(name='count')

fig_formation = px.bar(formation_counts,
                       x='team',
                       y='count',
                       color='formation',
                       text='count',
                       facet_col='result',
                       title='Frequência das Formaçőes Táticas por Equipe e Resultado',
                       labels={'team': 'Equipe', 'count': 'Frequência', 'formation': 'Formação', 'result': 'Resultado'},
                       color_discrete_map={'4-3-3': 'blue'},
                       facet_col_wrap=3,
                       height=600)

fig_formation.update_layout(
    xaxis_title='Equipe',
    yaxis_title='Frequência',
    xaxis_tickangle=-45,
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title='Formação',
    legend_title_font_size=16,
    legend_font_size=14,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

fig_formation.update_traces(texttemplate='%{text}', textposition='outside')

fig_formation.show()

## Posse de bola

Qual a média de posse de bola da equipe mandante quando ela perdeu o jogo?

Responda em forma de gráfico para poder comparar entre todas as equipes.

In [21]:
home_losses = data_cleaned[(data['venue'] == 'Home') & (data['result'] == 'L')]
average_possession_home_losses = home_losses.groupby('team')['poss'].mean().reset_index()

fig_possession = px.bar(average_possession_home_losses,
                        x='team',
                        y='poss',
                        title='Média de Posse de Bola da Equipe Mandante Quando Perdeu o Jogo',
                        labels={'team':'Equipe', 'poss':'Média de Posse de Bola (%)'})

fig_possession.show()

## Classificação

Qual a classificação final do campeonato?

Responda em forma de tabela, ordenada do primeiro colocado ao último.
Você deverá apresentar o brasão do time (pode ser obtido online [aqui](https://logodetimes.com/premier-league/), mas automatize a busca, não salve o logo localmente), o nome, a quantidade de pontos e a posição final no campeonato.

**OBS** Vitória vale 3 pontos, empates 1 ponto e derrota 0 pontos.

In [69]:
data_cleaned['points'] = data_cleaned['result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))

team_points = data_cleaned.groupby('team')['points'].sum().reset_index()
team_points = team_points.sort_values(by='points', ascending=False).reset_index(drop=True)

def get_team_logo(team_name):
    # Converte o nome da equipe para o padrão da URL
    formatted_team_name = team_name.lower().replace(' ', '-') + '-football-club.png'
    search_url = f"https://logodetimes.com/wp-content/uploads/{formatted_team_name}"
    response = requests.get(search_url)
    if response.status_code == 200:
        return search_url
    return None

team_points['logo'] = team_points['team'].apply(get_team_logo)

fig = go.Figure(data=[go.Table(
    header=dict(values=['Posição', 'Brasão', 'Equipe', 'Pontos'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[team_points.index + 1, team_points['logo'], team_points['team'], team_points['points']],
               fill_color='lavender',
               align='left',
               format=["", "<img src='{}'>", "", ""],
               height=30))
])

fig.update_layout(title="Classificação Final do Campeonato")
fig.show()

## Público

Crie uma visualização que apresente o público total de cada equipe, e também um detalhamento por jogos.

Apresentar os dados usando a técnica "DrillDown" será considerado um "plus". A visualização DrillDown permite que gráficos sejam detalhados com cliques em itens do gráfico.

Um exemplo com tabelas pode ser encontrado [aqui](https://dash.plotly.com/dash-ag-grid/enterprise-master-detail).

Um exemplo usando gráficos pode ser encontrado [aqui](https://community.plotly.com/t/show-and-tell-drill-down-functionality-in-dash-using-callback-context/54403?u=atharvakatre)

In [31]:
data_cleaned = data.dropna(subset=['attendance'])
total_attendance_per_team = data_cleaned.groupby('team')['attendance'].sum().reset_index()
team_names = total_attendance_per_team['team'].unique()

team_name_initial = total_attendance_per_team['team'].iloc[0]
team_games = get_team_games(team_name_initial)

def get_team_games(team_name):
    return data_cleaned[data_cleaned['team'] == team_name]

def update_game_detail(team_name):
    team_games = get_team_games(team_name)
    fig_combined.data[1].x = team_games['date']
    fig_combined.data[1].y = team_games['attendance']
    fig_combined.data[1].name = f'Público por Jogo - {team_name}'
    fig_combined.layout.annotations[1].text = f'Público por Jogo - {team_name}'

fig_total_attendance = px.bar(total_attendance_per_team,
                              x='team',
                              y='attendance',
                              title='Público Total por Equipe',
                              labels={'team': 'Equipe', 'attendance': 'Público Total'})

fig_team_games = px.bar(team_games,
                        x='date',
                        y='attendance',
                        title=f'Público por Jogo - {team_name_initial}',
                        labels={'date': 'Data do Jogo', 'attendance': 'Público'})

fig_combined = make_subplots(rows=2, cols=1,
                             subplot_titles=('Público Total por Equipe', f'Público por Jogo - {team_name_initial}'))

fig_combined.add_trace(go.Bar(x=total_attendance_per_team['team'],
                              y=total_attendance_per_team['attendance'],
                              name='Público Total por Equipe'), row=1, col=1)

fig_combined.add_trace(go.Bar(x=team_games['date'],
                              y=team_games['attendance'],
                              name=f'Público por Jogo - {team_name_initial}'), row=2, col=1)

dropdown_buttons = []
for team in team_names:
    dropdown_buttons.append(dict(method='restyle',
                                 label=team,
                                 args=[{'x': [get_team_games(team)['date']],
                                        'y': [get_team_games(team)['attendance']],
                                        'name': f'Público por Jogo - {team}'},
                                       [1]]))

fig_combined.update_layout(
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.5,
        'xanchor': 'center',
        'y': 1.15,
        'yanchor': 'top',
    }],
    height=700
)

fig_combined.show()